In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import os
import matplotlib.pyplot as plt
import argparse


from tokenizer import SimpleTokenizer
from dataset import SpeechesClassificationDataset, LanguageModelingDataset
from transformer import TransformerEncoder
from classifier import Classifier,SpeechClassifier

In [7]:
seed = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

""" Hyperparameters to use for training to roughly match 
the numbers mentioned in the assignment description """
batch_size = 16  # Number of independent sequences  we will process in parallel
block_size = 32  # Maximum context length for predictions
learning_rate = 1e-3  # Learning rate for the optimizer
n_embd = 64  # Embedding dimension
n_head = 2  # Number of attention heads
n_layer = 4  # Number of transformer layers


""" Hyperparameters for Language decoder. """
eval_interval = 100  # How often to evaluate train and test perplexity during training
max_iters = 500 # For language modeling, we can process all the batches for the entire dataset, but that takes a while, so we'll limit it to 500 iterations. For batch size of 16 and block size of  32, this is roughly, this is  500 * 16 * 32 = 256000 tokens, SOTA LMs are trained on trillions of tokens, so this is a very small dataset.
eval_iters = 200  # Number of iterations to evaluate perplexity on the test set


""" Classifier training hyperparameters. """
n_input = 64  # Input size for the classifier, should match the embedding size of the transformer
n_hidden = 100  # Hidden size for the classifier
n_output = 3  # Output size for the classifier, we have 3 classes
epochs_CLS = 15 # epochs for classifier training

In [8]:
def load_texts(directory):
    """
    This function loads all texts from the specified directory, ignoring any files with "test" in their name. The text is used for "training" the tokenizer. Since our tokenizer is simple, we don't need to do any training, but we still need to ignore the test data. 
    """

    texts = []
    files = os.listdir(directory)
    for filename in files: 
        if "test" in filename:  ## don't "read test files"
            continue
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            texts.append(file.read())
    return texts


def collate_batch(batch):
    """ Collate a batch of data into a single tensor with padding."""
    data, labels = zip(*batch)  # Separate the data and labels
    # Pad sequences to the fixed length
    padded_sequences = pad_sequence(data, batch_first=True, padding_value=0)
    padded_sequences = padded_sequences[:, :block_size]  # Truncate if longer
    # Add padding if shorter
    padded_sequences = torch.nn.functional.pad(padded_sequences, (0, max(0, block_size - padded_sequences.shape[1])), "constant", 0)
    labels = torch.stack(labels)  
    return padded_sequences, labels

In [9]:
print("Loading data and creating tokenizer ...")
texts = load_texts('speechesdataset')
tokenizer = SimpleTokenizer(' '.join(texts)) # create a tokenizer from the data
print("Vocabulary size is", tokenizer.vocab_size)

train_CLS_dataset = SpeechesClassificationDataset(tokenizer, "speechesdataset/train_CLS.tsv")
train_CLS_loader = DataLoader(train_CLS_dataset, batch_size=batch_size,collate_fn=collate_batch,shuffle=True)
test_CLS_dataset = SpeechesClassificationDataset(tokenizer, "speechesdataset/test_CLS.tsv")
test_CLS_loader = DataLoader(test_CLS_dataset, batch_size=batch_size,collate_fn=collate_batch,shuffle=False)

Loading data and creating tokenizer ...
Vocabulary size is 5755


In [35]:
print(type(texts[0][:32]))

<class 'str'>


In [2]:
inputfile = "speechesdataset/train_LM.txt"
with open(inputfile, 'r', encoding='utf-8') as f:
    lmtrainText = f.read()

In [4]:
lmtrainText

'It is costly and politically difficult to continue this conflict.\nMore of you have lost your homes and even more are watching your home values plummet.\nThrough this remarkable chapter in the history of the United States and Iraq, we have met our responsibility.\nOur relations abroad were strained.\nYou can tell that I\'m a lame duck because nobody\'s following instructions.\nAnd today, as my call for a time frame to remove our troops from Iraq has been echoed by the Iraqi government and even the Bush Administration, even after we learned that Iraq has a $79 billion surplus while we\'re wallowing in deficits, John McCain stands alone in his stubborn refusal to end a misguided war.\nOver the course of these 8 years, I\'ve seen the hopeful faces of young graduates and our newest military officers.\nBut most of all, congratulations to the Class of 2016!\nThe pace of change is not subsiding; it is accelerating.\nAmerica, we cannot turn back.\nBut for those of us fortunate enough to have 

In [19]:
import torch

mask = torch.triu(torch.ones(32, 32), diagonal=1).bool()
mask = (~mask).float().unsqueeze(0)
mask.shape

torch.Size([32, 32])